# Infraestructure Manager revenue maximization with GSA

## 0. Load libraries

In [43]:
%load_ext autoreload
%autoreload 2

import numpy as np

from src.entities import GSA

from copy import deepcopy
from math import cos, pi, e
from typing import List, Mapping, Union

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Define corridor

As an example, we will use the Spanish south high-speed railway corridor.



In [4]:
# Define corridor

corridor = {"MAD": {
                "CIU": {
                    "COR": {
                        "SEV": {
                            "CAD": {}
                        },
                        "PGE": {
                            "ANT": {
                                "GRA": {},
                                "MAL": {}
                                    }
                                }
                            }
                        }
                    }
            }

### 1.1. Get lines from corridor

In [5]:
# Get lines from corridor

def get_lines(corridor: dict, path: Union[list, None]=None) -> list:
    """
    Get all the lines in the corridor
    
    Args:
        corridor (dict): dictionary with the corridor structure
        path (list, optional): list of nodes
    
    Returns:
        list of lines
    """
    if path is None:
        path = []

    lines = []
    for node, child in corridor.items():
        new_path = path + [node]
        if not child:  # If the node has no children, it is a leaf
            lines.append(new_path)
        else:
            lines.extend(get_lines(child, new_path))  # If the node has children, we call the function recursively

    return lines

get_lines(corridor)

[['MAD', 'CIU', 'COR', 'SEV', 'CAD'],
 ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'GRA'],
 ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'MAL']]

### 1.2. Sample a random line, and a random route from the line

In [6]:
def sample_line(lines: list) -> list:
    """
    Sample a random line from the list of lines
    
    Args:
        lines (list): list of lines
    
    Returns:
        list: random line
    """
    return lines[np.random.randint(len(lines))] 

def sample_route(line: list) -> list:
    """
    Sample a random route from line
    
    Args:
        line (list): list of stations
        
    Returns:
        list: random route
    """
    return line[np.random.randint(0, len(line)-1):]

lines = get_lines(corridor)

line = sample_line(lines)
print(f"Sampled line: {line}")

# Sample a random route from line (at least two stations)
route = sample_route(line)
print(f"Sampled route: {route}")

Sampled line: ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'GRA']
Sampled route: ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'GRA']


### 1.3 Generate random timetable for a route

- Times in minutes.
- Initial time is randomized between 0 and 24*60 (24 hours).
- The time between stations is also randomized between 30 and 120 minutes.
- The time to dwell at each station is randomized between 2 and 8 minutes.

In [7]:
def get_timetable(route: list) -> dict:
    """
    Generate random timetable for route r
    
    Args:
        route (list): list of stations
    
    Returns:
        dict: timetable
    """
    timetable = {}
    AT = np.random.randint(0, 24*60)
    DT = AT
    for i, sta in enumerate(route):
        if i == 0 or i == len(route)-1:
            timetable[sta] = (AT, AT)
        else:
            timetable[sta] = (AT, DT)
            
        AT += np.random.randint(30, 120)
        DT = AT + np.random.randint(2, 8)
        
    return timetable

get_timetable(route)

{'MAD': (856, 856),
 'CIU': (943, 950),
 'COR': (1031, 1036),
 'PGE': (1102, 1109),
 'ANT': (1167, 1174),
 'GRA': (1268, 1268)}

## 2. Generate as many requested services as needed

In [8]:
# Generate random requested timetable in corridor for a day t

def get_schedule_request(n_services: int) -> dict:
    """
    Generate random timetable
    
    Args:
        n_services (int): number of services
    
    Returns:
        dict: timetable
    """
    return {i: get_timetable(sample_route(sample_line(lines))) for i in range(1, n_services+1)}

# Generate random schedule
schedule = get_schedule_request(3)
schedule

{1: {'MAD': (955, 955),
  'CIU': (1009, 1015),
  'COR': (1071, 1075),
  'PGE': (1149, 1154),
  'ANT': (1255, 1260),
  'MAL': (1302, 1302)},
 2: {'COR': (409, 409), 'SEV': (515, 522), 'CAD': (587, 587)},
 3: {'CIU': (443, 443),
  'COR': (532, 538),
  'PGE': (598, 605),
  'ANT': (692, 694),
  'MAL': (751, 751)}}

## 3. Define feasibility function

A service $ i $ is feasible (can be scheduled and, therefore, $ S_i $ take the value 1) if the following condition is met:

$ | DT_{ij} - DT_{kj} | \geq security\_gap $

Where:
- $ DT_{ij} $ is the departure time for service $ i $ from station $ j $.
- $  DT_{kj} $ is the departure time for service $ k $ from the same station $ j $.

Inequality must be met for all services ($ k \neq j $) that also operate at station $ j $ for service $ i $ to be feasible.

1) Requests from the Railway Undertakings (RUs) are considered as a set of services that can be scheduled or not. The decision variable $ S_i $ is binary and takes the value 1 if the service $ i $ is scheduled and 0 otherwise.

Service ID |  RU   |          Trip          |              Requested schedule              | 
:---: |:-----:|:----------------------:|:--------------------------------------------:|
1 | Renfe |   `[MAD, BAR, FIG]`    |      `[(0, 0), (148, 152), (180, 180)]`      |
2 | Ouigo | `[MAD, ZAR, BAR, FIG]` | `[(8, 8), (28, 30), (165, 167), (210, 210)]` |
3 | Iryo  |   `[MAD, BAR, FIG]`    |     `[(30, 30), (180, 182), (225, 225)]`     |


2) Get conflict matrices for each service. A conflict matrix is a binary matrix of 0's and 1's. A 1 appears in the outputs of services (different from the one we are analyzing) that interferes with a station of the service in question.

Service 1: Has an interference with the service 2 at the departure from Madrid.

Service ID | MAD | BAR | FIG |
:---: | :---: | :---: | :---: |
1 | 0 | 0 | 0 |
2 | 1 | 0 | 0|
3 | 0 | 0 | 0 |

In order to be feasible, the following condition must be met:

$ R_{S_1} = (S_1 \cdot 0 + S_1 \cdot 1 + S_3 \cdot 0) + (S_1 \cdot 0 + S_1 \cdot 0 + S_3 \cdot 0) +(S_1 \cdot 0 + S_1 \cdot 1 + S_3 \cdot 0) = 0 $ \\

Service 2: Has an interference with the service 1 at the departure from Madrid.

ID servicio | MAD | ZAR | BAR | FIG |
:---: | :---: | :---: | :---: | :---: |
1 | 0 | 0 | 0 | 0 |
2 | 1 | 0 | 0 | 0|
3 | 0 | 0 | 0 | 0 |

In order to be feasible, the following condition must be met:

$ R_{S_2} = (S_1 \cdot 1 + S_1 \cdot 0 + S_3 \cdot 0) + (S_1 \cdot 0 + S_1 \cdot 0 + S_3 \cdot 0) +(S_1 \cdot 0 + S_1 \cdot 1 + S_3 \cdot 0) = 0 $ \\

Service 3: Does not have any interference with the other services.

ID servicio | MAD | BAR | FIG |
:---: |:---:| :---: | :---: |
1 |  0  | 0 | 0 |
2 |  0  | 0 | 0|
3 |  0  | 0 | 0 |

In order to be feasible, the following condition must be met:

$ R_{S_2} = (S_1 \cdot 0 + S_1 \cdot 0 + S_3 \cdot 0) + (S_1 \cdot 0 + S_1 \cdot 0 + S_3 \cdot 0) +(S_1 \cdot 0 + S_1 \cdot 1 + S_3 \cdot 0) \leq 0 $ \\

We can define a general restriction to check if the timetable is feasible or not:

$ S_1 \cdot R_{S_1} + S_2 \cdot R_{S_2} + S_2 \cdot R_{S_2} \leq 0$

This equation can be read as follows:

- If a service $ S_i $ is planned, then its restrictions $ R_{S_i} $ must be met. If the service $ S_i $ is not planned, its own interferences do not have to be taken into account.
- For the set of schedules to be feasible, there must be no conflict between any of the planned services (all the $ S_i $ that take the value $ 1 $)

In [135]:
class RevenueMaximization:
    """
    Class for the revenue maximization problem.
    """
    def __init__(self,
                 requested_schedule: Mapping[int, Mapping[str, Union[int, int]]],
                 revenue: Mapping[int, Mapping[str, Union[float, int]]],
                 safe_headway: int=10
                 ) -> None:
        """
        Constructor
        
        Args:
            requested_schedule (dict): requested schedule
        """
        self.requested_schedule = requested_schedule 
        self.revenue = revenue
        self.updated_schedule = deepcopy(requested_schedule)
        self.requested_times = self.get_real_vars()
        self.real_boundaries = self.get_real_boundaries()
        self.discrete_boundaries = np.array([(0, 1) for _ in range(len(requested_schedule))])
        self.boundaries = {'real': self.real_boundaries, 'discrete': self.discrete_boundaries}
        self.safe_headway = safe_headway
    
    @staticmethod
    def penalty_function(x: float, k: int) -> float:
        """
        Penalty function
        
        Args:
            x (float): x
            k (int): k
        
        Returns:
            float: penalty
        """
        return 1 - e**(-k * x**2) * ( (1/2) * cos(pi * x) + (1/2) )
    
    def get_revenue(self, solution: Mapping[str, List[Union[float, int]]]) -> int:
        """
        Get IM revenue.
        
        Args:
            solution (dict): solution
        
        Returns:
            float: IM revenue
        """
        self.update_schedule(solution)
        S_i = np.array(solution["discrete"])
        
        im_revenue = 0
        for i, service in enumerate(self.requested_schedule):
            k = self.revenue[service]['k']
            departure_station = list(self.requested_schedule[service].keys())[0]
            departure_time_delta = abs(self.updated_schedule[service][departure_station][1] - self.requested_schedule[service][departure_station][1])
            tt_penalties = []
            for j, stop in enumerate(self.requested_schedule[service].keys()):
                if i == 0 or i == len(self.requested_schedule[service]) - 1:
                    continue
                tt_penalty = self.penalty_function(abs(self.updated_schedule[service][stop][1] - self.requested_schedule[service][stop][1]) / self.safe_headway, k)
                tt_penalties.append(tt_penalty * self.revenue[service]['tt_max_penalty'])
            dt_penalty = self.penalty_function(departure_time_delta / self.safe_headway, k) * self.revenue[service]['dt_max_penalty']
            im_revenue += self.revenue[service]['canon'] * S_i[i] - dt_penalty - np.sum(tt_penalties)
            
        return im_revenue
    
    def update_schedule(self, solution):
        """
        Update schedule with the solution
        
        Args:
            solution (list): solution
        """
        times = solution["real"] if solution["real"].size else self.get_real_vars()
        s_idx = 0
        for i, service in enumerate(self.updated_schedule):
            if solution["discrete"][i] != 0:
                for j, stop in enumerate(self.updated_schedule[service]):
                    if j == 0 or j == len(self.updated_schedule[service]) - 1:
                        self.updated_schedule[service][stop] = (times[s_idx], times[s_idx])
                        s_idx += 1
                    else:
                        self.updated_schedule[service][stop] = (times[s_idx], times[s_idx+1])
                        s_idx += 2
            else:
                for j, stop in enumerate(self.updated_schedule[service]):
                    if j == 0 or j == len(self.updated_schedule[service]) - 1:
                        self.updated_schedule[service][stop] = (self.requested_times[s_idx], self.requested_times[s_idx])
                        s_idx += 1
                    else:
                        self.updated_schedule[service][stop] = (self.requested_times[s_idx], self.requested_times[s_idx+1])
                        s_idx += 2
    
    def _departure_time_feasibility(self, solution) -> bool:
        """
        Check if there are any conflicts with the departure times.
        
        Args:
            solution (dict): solution
            security_gap (int, optional): security gap
        
        Returns:
            bool: True if the departure time is feasible, False otherwise
        """
        S_i = np.array(solution["discrete"])
        security_array = []

        # Get conflicts between services
        for service in self.updated_schedule:
            service_sec_arr = []  # Build security matrix for service (columns are stops, rows are other services)
            for service_k in self.updated_schedule:
              service_sec_row = [] 
              for stop in self.updated_schedule[service]:
                if service_k == service or stop not in self.updated_schedule[service_k]:
                  service_sec_row.append(0)
                  continue

                if abs(self.updated_schedule[service][stop][1] - self.updated_schedule[service_k][stop][1]) < self.safe_headway:
                  service_sec_row.append(1)
                else:
                  service_sec_row.append(0)
              service_sec_arr.append(service_sec_row)

            security_array.append(np.array(service_sec_arr))
            
        # Get array of conflicts. If there is a conflict, the dot product will be different from zero
        if not S_i.dot(np.array([np.sum(S_i.dot(ssa)) for ssa in security_array])):
            return True
        return False
    
    def _stop_times_feasibility(self, solution):
        """
        Check if the travel times are feasible. In order to be feasible, the travel times must be greater than the requested ones.
        
        Returns:
            bool: True if the travel times are feasible, False otherwise
        """
        S_i = np.array(solution["discrete"])
        for i, service in enumerate(self.requested_schedule):
            if S_i[i] == 0:
                continue
            original_service_times = tuple(self.requested_schedule[service].values())
            updated_service_times = tuple(self.updated_schedule[service].values())
            for j in range(1, len(original_service_times)-1):
                original_st = original_service_times[j][1] - original_service_times[j][0]
                updated_st = updated_service_times[j][1] - updated_service_times[j][0]
                if updated_st < original_st:
                    return False
        return True
        
    def _travel_times_feasibility(self, solution) -> bool:
        """
        Check if the travel times are feasible. In order to be feasible, the travel times must be greater than the requested ones.
        
        Returns:
            bool: True if the travel times are feasible, False otherwise
        """
        S_i = np.array(solution["discrete"])
        for i, service in enumerate(self.requested_schedule):
            if S_i[i] == 0:
                continue
            original_service_times = tuple(self.requested_schedule[service].values())
            updated_service_times = tuple(self.updated_schedule[service].values())
            for j in range(len(original_service_times)-1):
                original_tt = original_service_times[j+1][0] - original_service_times[j][1]
                updated_tt = updated_service_times[j+1][0] - updated_service_times[j][1]
                if updated_tt < original_tt:
                    return False
        return True
            
    def is_feasible(self, solution) -> bool:
        """
        Check if the solution is feasible
        
        Args:
            solution (dict): solution
            security_gap (int, optional): security gap
        
        Returns:
            bool: True if the solution is feasible, False otherwise
        """
        self.update_schedule(solution)
        
        if self._departure_time_feasibility(solution) and self._travel_times_feasibility(solution) and self._stop_times_feasibility(solution):
            return True
        return False
    
    def get_real_vars(self):
        """
        Get real variables
        
        Returns:
            list: real variables with requested schedule times.
        """
        real_vars = []
        for service in self.requested_schedule:
            for i, stop in enumerate(self.requested_schedule[service]):
                if i == 0 or i == len(self.requested_schedule[service]) - 1:
                    real_vars.append(self.requested_schedule[service][stop][0])
                else: 
                    real_vars.append(self.requested_schedule[service][stop][0])
                    real_vars.append(self.requested_schedule[service][stop][1])

        return real_vars
    
    def get_real_boundaries(self):
        """
        Get real boundaries
        
        Returns:
            list: real boundaries
        """
        real_vars = self.get_real_vars()
        real_boundaries = []
        for rv in real_vars:
            real_boundaries.append((rv-10, rv+10))

        return np.array(real_boundaries)

In [126]:
schedule = get_schedule_request(20)

In [147]:
# Dummy schedule
schedule = {1: {'MAD': (0, 0), 'BAR': (148, 152), 'FIG': (180, 180)},
            2: {'MAD': (8, 8), 'ZAR': (28, 30), 'BAR': (165, 167), 'FIG': (210, 210)},
            3: {'MAD': (30, 30), 'BAR': (180, 182), 'FIG': (225, 225)}}

In [148]:
from scipy.stats import loguniform

np.random.seed(seed=22)

def get_revenue_behaviour(schedule: dict) -> dict:
    """
    Get revenue behaviour
    
    Args:
        schedule (dict): schedule
    
    Returns:
        dict: revenue behaviour
    """
    revenue = {}
    bias = [0.2, 0.35, 0.1]
    for service in schedule:
        b = np.random.choice(bias)
        base_price = 55 * len(schedule[service])
        canon = base_price + b * base_price
        k = loguniform.rvs(0.01, 100, 1)
        max_penalty = canon * 0.4
        dt_penalty = max_penalty * 0.35
        tt_penalty = (max_penalty - dt_penalty) / (len(schedule[service]) - 1)
        revenue[service] = {'canon': canon, 'k': k, 'dt_max_penalty': dt_penalty, 'tt_max_penalty': tt_penalty}
    return revenue

revenue = get_revenue_behaviour(schedule)
revenue

{1: {'canon': 222.75,
  'k': 7.424686268142557,
  'dt_max_penalty': 31.185000000000002,
  'tt_max_penalty': 19.305000000000003},
 2: {'canon': 264.0,
  'k': 1.4810078247291318,
  'dt_max_penalty': 36.96,
  'tt_max_penalty': 17.160000000000004},
 3: {'canon': 181.5,
  'k': 14.186717826602765,
  'dt_max_penalty': 25.41,
  'tt_max_penalty': 15.730000000000004}}

In [149]:
sm = RevenueMaximization(schedule, revenue, safe_headway=10)

In [163]:
gsa_algo = GSA(objective_function=sm.get_revenue,
               is_feasible=sm.is_feasible,
               r_dim=len(sm.real_boundaries),
               d_dim=len(sm.requested_schedule),
               boundaries=sm.boundaries)

In [164]:
gsa_algo.set_seed(seed=28)

training_history = gsa_algo.optimize(population_size=20,
                                     iters=100,
                                     chaotic_constant=False,
                                     repair_solution=True)

Initializing positions of the individuals in the population...
Positions of the individuals in the population successfully initialized!!
Initial population: {'real': array([[ 2.29319663e+02,  2.21284936e+02,  2.33925800e+02,
         2.25168486e+02,  2.17558607e+02,  2.20143196e+02,
         2.28665953e+02,  2.28576095e+02,  2.22133226e+02,
         2.22257564e+02,  2.32514818e+02,  2.34661316e+02,
         2.33242294e+02,  2.29416798e+02],
       [ 2.28489881e+02,  2.27302500e+02,  2.19549143e+02,
         2.25064807e+02,  2.24843286e+02,  2.16540145e+02,
         2.17351397e+02,  2.23922868e+02,  2.18416089e+02,
         2.25448107e+02,  2.16094934e+02,  2.31439339e+02,
         2.34043418e+02,  2.28416708e+02],
       [ 2.27311428e+02,  2.17981789e+02,  2.18585153e+02,
         2.34270129e+02,  2.20623988e+02,  2.30406646e+02,
         2.18889889e+02,  2.31795004e+02,  2.19712897e+02,
         2.27732487e+02,  2.27276639e+02,  2.32765558e+02,
         2.20628527e+02,  2.31409692e+02

In [76]:
gsa_algo = GSA(objective_function=sm.get_num_trains,
               is_feasible=sm.is_feasible,
               r_dim=0,
               d_dim=len(sm.requested_schedule),
               boundaries=sm.boundaries)

In [77]:
gsa_algo.set_seed(seed=28)

training_history = gsa_algo.optimize(population_size=20,
                                     iters=300,
                                     chaotic_constant=False,
                                     repair_solution=True)

Initializing positions of the individuals in the population...
Positions of the individuals in the population successfully initialized!!
GSA is optimizing  "get_num_trains"
####################################################################################################
REPAIRING SOLUTION... 
####################################################################################################
####################################################################################################
SOLUTION SUCCESSFULLY REPAIRED!!
####################################################################################################
####################################################################################################
REPAIRING SOLUTION... 
####################################################################################################
####################################################################################################
SOLUTION SUCCESSFULLY REPAIRED!!
########